In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

ModuleNotFoundError: No module named 'mlxtend'

In [ ]:
df =  pd.read_parquet('logs.parquet')

# Reglas de asociación

## Filtrado y exploracion de los datos

Observación de cuantas piezas unicas hay en nuestro conjunto, descartando aquellas a las que pertenecen a un diseño no válido, y cuantas operaciones han habido.

In [ ]:
añadir_pieza = df[(df['Evento'] == 'AÑADIR_PIEZA') & (df['diseñoid'] >= 1)].copy()
añadir_pieza = añadir_pieza[['pieza','diseñoid']]
print(añadir_pieza.head(10))
print(f"Número total de transacciones: {añadir_pieza['diseñoid'].nunique()}")
print(f"Número total de ítems (productos): {añadir_pieza['pieza'].nunique()}")

## Observación del tamaño de las transacciones

In [ ]:
print(añadir_pieza.groupby('diseñoid')['pieza'].size().describe(percentiles=[.25, .5, .75, .9]))
fig, ax = plt.subplots(figsize=(7, 3))
añadir_pieza.groupby('diseñoid')['pieza'].size().plot.hist(ax=ax)
ax.set_title('Distribución del tamaño de las transacciones');
ax.set_xlabel('Número de ítems');

Podemos observar como la mayoría de los usuarios escogen entre 3 y 17 piezas, siendo una media de 12 piezas por diseño, además el 90% escogen menos de 29 piezas  

## Transformacion del dataframe en transacciones

In [ ]:
transacciones = añadir_pieza.groupby('diseñoid')['pieza'].apply(list).to_list()
te = TransactionEncoder()
te_ary = te.fit(transacciones).transform(transacciones)
df_transacciones = pd.DataFrame(te_ary, columns=te.columns_)
df_transacciones.head(5)

## Aplicación del algoritmo apriori

Uso un soporte mínimio del 0.003 para que me genere las suficientes reglas para poder trabajar con ellas.

In [ ]:
itemsets = apriori(df_transacciones, min_support=0.003, use_colnames=True)

Ordeno los itemsets por frecuencia de manera descendente

In [ ]:

itemsets['n_items'] = itemsets['itemsets'].apply(len)
itemsets.sort_values(by='support', ascending=False)

Filtro los itemsets que únicamente contienen un item

In [ ]:
itemsets.query('n_items >= 2').sort_values('support', ascending=False)

### Generación de las Reglas de Asociación

In [ ]:
reglas = association_rules(itemsets, metric="confidence", min_threshold=0.1)
columnas = ['antecedents', 'consequents','support','confidence','lift']
reglas = reglas[columnas]
reglas.tail(10)


### Filtrado de reglas

In [ ]:
reglas_support = reglas[reglas['support'] > 0.003]
reglas_support

In [ ]:
reglas_antecentes = reglas_support[reglas_support['antecedents'].apply(lambda x: len(x) > 1)]
reglas_antecentes.head(10)

In [ ]:
reglas_antecentes.sort_values(by = 'lift', ascending = False).head(10)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(reglas['support'], reglas['confidence'], alpha=0.5)
plt.title('Soporte vs. Confianza')
plt.xlabel('Soporte')
plt.ylabel('Confianza')
plt.show()

In [ ]:
# Visualizar soporte vs lift o confianza vs lift
reglas.nunique()

## Recomendador de piezas

In [ ]:
def recomendar_piezas(piezas_usuario, reglas, top_n=5):
    reglas_validas = reglas[reglas['antecedents'].apply(lambda x: x.issubset(piezas_usuario))]
    
    reglas_validas = reglas_validas.sort_values(['confidence', 'lift'], ascending=False)
    

    piezas_recomendadas = []
    for consequents in reglas_validas['consequents']:
        for pieza in consequents:
            if pieza not in piezas_usuario and pieza not in piezas_recomendadas:
                piezas_recomendadas.append(pieza)
                if len(piezas_recomendadas) >= top_n:
                    return piezas_recomendadas
    return piezas_recomendadas

In [ ]:
piezas_usuario = {'ED-PPC-02'}
recomendaciones = recomendar_piezas(piezas_usuario, reglas, top_n=6)

print("Piezas recomendadas:", recomendaciones)